In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import eval.eval as ev
import process.file as f
import process.atlas as atl
import utils.utils as uti
import visualize.visualizer as vis
import os 
import scipy
from scipy import io as sio, ndimage
import tifffile

In [ ]:
atlas = atl.Atlas()
neuron_dict = atlas.create_dictionary()
df_atlas = atlas.get_df()

In [35]:
%matplotlib qt

def comp_acc_align():

    labels = []
    per_IDs = []
    per_corrects = []
    per_corr_cols = []
    per_corr_poss = []
    per_corr_boths = []
    cost = []
    cost_col = []
    cost_pos = []
    cost_both = []

    for folder in os.listdir('data/NP_FOCO_cropped'):
        if not folder[0:2] == '20':
            continue 

        df_norm = f.proc_FOCO('data/NP_FOCO_cropped/'+folder)
        df_color = f.proc_FOCO('data/NP_FOCO_set_color/'+folder)
        df_pos = f.proc_FOCO('data/NP_FOCO_set_pos/'+folder)
        df_both = f.proc_FOCO('data/NP_FOCO_set_both/'+folder)

        per_ID, per_correct, per_top2, correctID, correcttop2 = uti.check_accuracy(df_norm)
        per_ID_color, per_correct_color, per_top2_color, correctID_color, correcttop2_color = uti.check_accuracy(df_color)
        per_ID_pos, per_correct_pos, per_top2_pos, correctID_pos, correcttop2_pos = uti.check_accuracy(df_pos)
        per_ID_both, per_correct_both, per_top2_both, correctID_both, correcttop2_both = uti.check_accuracy(df_both)

        cost_xyz, cost_rgb = uti.calc_costs(df_atlas, atlas.sigma, df_norm)
        cost_xyz_col, cost_rgb_col = uti.calc_costs(df_atlas, atlas.sigma, df_color)
        cost_xyz_pos, cost_rgb_pos = uti.calc_costs(df_atlas, atlas.sigma, df_pos)
        cost_xyz_both, cost_rgb_both = uti.calc_costs(df_atlas, atlas.sigma, df_both)

        labels.append(folder[:-4])
        per_IDs.append(per_ID)
        per_corrects.append(per_correct)
        per_corr_cols.append(per_correct_color)
        per_corr_poss.append(per_correct_pos)
        per_corr_boths.append(per_correct_both)
        cost.append([cost_xyz, cost_rgb])
        cost_col.append([cost_xyz_col, cost_rgb_col])
        cost_pos.append([cost_xyz_pos, cost_rgb_pos])
        cost_both.append([cost_xyz_both, cost_rgb_both])

    cost = np.asarray(cost)
    cost_col = np.asarray(cost_col)
    cost_pos = np.asarray(cost_pos)
    cost_both = np.asarray(cost_both)

    fig, ax = plt.subplots(2,1)

    x = np.arange(len(labels))
    width = 0.6

    ax[0].bar(x-3*width/8, per_corrects, width = width/4, label ='Percent correct original data', edgecolor= 'black')
    ax[0].bar(x-width/8, per_corr_cols, width =width/4, label = 'Percent correct color set data', edgecolor= 'black')
    ax[0].bar(x+width/8, per_corr_poss, width =width/4, label = 'Percent correct position set data', edgecolor= 'black')
    ax[0].bar(x+3*width/8, per_corr_boths, width = width/4, label = 'Percent correct both set data', edgecolor= 'black')
    ax[0].set_ylabel('Percent accuracy')
    ax[0].set_title('Comparing accuracy from different data processing approaches')
    ax[0].legend(loc='upper center', ncol=2)

    ax[0].set_xticks(x)
    ax[0].set_xticklabels(labels)
    ax[0].tick_params(labelrotation=45)
    ax[0].set_ylim(0, 1.5)
    ax[0].axhline(y = 1, color = 'black', linestyle = '--')


    ax[1].bar(x-3*width/8, cost[:,0], width = width/4, label ='xyz alignment', color= 'purple', edgecolor='black')
    ax[1].bar(x-width/8, cost_col[:,0], width =width/4, color= 'purple', edgecolor='black')
    ax[1].bar(x+width/8, cost_pos[:,0], width =width/4, color= 'purple', edgecolor='black')
    ax[1].bar(x+3*width/8, cost_both[:,0], width = width/4, color= 'purple', edgecolor='black')
    ax[1].bar(x-3*width/8, cost[:,1], bottom= cost[:,0], width = width/4, label ='rgb alignment', color = 'pink', edgecolor='black')
    ax[1].bar(x-width/8, cost_col[:,1], bottom= cost_col[:,0], width =width/4, color = 'pink', edgecolor='black')
    ax[1].bar(x+width/8, cost_pos[:,1], bottom = cost_pos[:,0], width =width/4, color = 'pink', edgecolor='black')
    ax[1].bar(x+3*width/8, cost_both[:,1], bottom =cost_both[:,0], width = width/4, color = 'pink', edgecolor='black')
    ax[1].set_ylabel('Alignment cost')
    ax[1].set_title('Comparing alignment cost from different data processing approaches')
    ax[1].legend(loc='upper left', ncol=2)

    ax[1].set_xticks(x)
    ax[1].set_xticklabels(labels)
    ax[1].tick_params(labelrotation=45)

    plt.tight_layout()

    plt.show()

comp_acc_align()

In [49]:
%matplotlib qt

def comp_all():

    labels = []
    per_IDs = []
    per_corrects = []
    per_corr_cols = []
    per_corr_poss = []
    per_corr_boths = []
    per_corr_hist = []
    per_corr_trans = []
    cost = []
    cost_col = []
    cost_pos = []
    cost_both = []
    cost_hist = []
    cost_trans = []

    for folder in os.listdir('data/NP_FOCO_cropped'):
        if not folder[0:2] == '20':
            continue 

        df_norm = f.proc_FOCO('data/NP_FOCO_median/'+folder)
        df_color = f.proc_FOCO('data/NP_FOCO_median_copy/'+folder)
        df_pos = f.proc_FOCO('data/NP_FOCO_hist_med/'+folder)
        df_both = f.proc_FOCO('data/NP_FOCO_hist_med_copy/'+folder)
        df_hist = f.proc_FOCO('data/NP_FOCO_trans_med/'+folder)
        df_trans = f.proc_FOCO('data/NP_FOCO_trans_med/'+folder)

        per_ID, per_correct, per_top2, correctID, correcttop2 = uti.check_accuracy(df_norm)
        per_ID_color, per_correct_color, per_top2_color, correctID_color, correcttop2_color = uti.check_accuracy(df_color)
        per_ID_pos, per_correct_pos, per_top2_pos, correctID_pos, correcttop2_pos = uti.check_accuracy(df_pos)
        per_ID_both, per_correct_both, per_top2_both, correctID_both, correcttop2_both = uti.check_accuracy(df_both)
        per_ID_hist, per_correct_hist, per_top2_hist, correctID_hist, correcttop2_hist = uti.check_accuracy(df_hist)
        per_ID_trans, per_correct_trans, per_top2_trans, correctID_trans, correcttop2_trans = uti.check_accuracy(df_trans)

        cost_xyz, cost_rgb = uti.calc_costs(df_atlas, atlas.sigma, df_norm)
        cost_xyz_col, cost_rgb_col = uti.calc_costs(df_atlas, atlas.sigma, df_color)
        cost_xyz_pos, cost_rgb_pos = uti.calc_costs(df_atlas, atlas.sigma, df_pos)
        cost_xyz_both, cost_rgb_both = uti.calc_costs(df_atlas, atlas.sigma, df_both)
        cost_xyz_hist, cost_rgb_hist = uti.calc_costs(df_atlas, atlas.sigma, df_hist)
        cost_xyz_trans, cost_rgb_trans = uti.calc_costs(df_atlas, atlas.sigma, df_trans)

        labels.append(folder[:-4])
        per_IDs.append(per_ID)
        per_corrects.append(per_correct)
        per_corr_cols.append(per_correct_color)
        per_corr_poss.append(per_correct_pos)
        per_corr_boths.append(per_correct_both)
        per_corr_hist.append(per_correct_hist)
        per_corr_trans.append(per_correct_trans)
        cost.append([cost_xyz, cost_rgb])
        cost_col.append([cost_xyz_col, cost_rgb_col])
        cost_pos.append([cost_xyz_pos, cost_rgb_pos])
        cost_both.append([cost_xyz_both, cost_rgb_both])
        cost_hist.append([cost_xyz_hist, cost_rgb_hist])
        cost_trans.append([cost_xyz_trans, cost_rgb_trans])

    cost = np.asarray(cost)
    cost_col = np.asarray(cost_col)
    cost_pos = np.asarray(cost_pos)
    cost_both = np.asarray(cost_both)

    fig, ax = plt.subplots(2,1)

    x = np.arange(len(labels))
    width = 0.6

    ax[0].bar(x-3*width/8, per_corrects, width = width/4, label ='Percent correct original data', edgecolor= 'black')
    ax[0].bar(x-width/8, per_corr_cols, width =width/4, label = 'Percent correct color set data', edgecolor= 'black')
    ax[0].bar(x+width/8, per_corr_poss, width =width/4, label = 'Percent correct position set data', edgecolor= 'black')
    ax[0].bar(x+3*width/8, per_corr_boths, width = width/4, label = 'Percent correct both set data', edgecolor= 'black')
    ax[0].set_ylabel('Percent accuracy')
    ax[0].set_title('Comparing accuracy from different data processing approaches')
    ax[0].legend(loc='upper center', ncol=2)

    ax[0].set_xticks(x)
    ax[0].set_xticklabels(labels)
    ax[0].tick_params(labelrotation=45)
    ax[0].set_ylim(0, 1.5)
    ax[0].axhline(y = 1, color = 'black', linestyle = '--')


    ax[1].bar(x-width/3, per_corrects, width = width/3, label ='Percent correct original data', color= 'purple', edgecolor='black')
    ax[1].bar(x, per_corr_hist, width =width/3, label= 'Percent correct equalized histogram', edgecolor='black')
    ax[1].bar(x+width/3, per_corr_trans, width =width/3, label= 'Percent correct transformed and equalized', edgecolor='black')
    ax[1].set_ylabel('Percent accuracy')
    ax[1].legend(loc='upper left', ncol=2)

    ax[1].set_xticks(x)
    ax[1].set_xticklabels(labels)
    ax[1].tick_params(labelrotation=45)

    plt.tight_layout()

    plt.show()

comp_all()

In [57]:
%matplotlib qt

def comp_hist_trans():

    labels = []
    per_IDs = []
    per_corrects = []
    per_corr_hist = []
    per_corr_trans = []
    cost = []
    cost_hist = []
    cost_trans = []

    for folder in os.listdir('data/NP_FOCO_cropped'):
        if not folder[0:2] == '20':
            continue 

        df_norm = f.proc_FOCO('data/NP_FOCO_cropped/'+folder)
        df_hist = f.proc_FOCO('data/NP_FOCO_hist_med/'+folder)
        df_trans = f.proc_FOCO('data/NP_FOCO_hist_match/'+folder)

        per_ID, per_correct, per_top2, correctID, correcttop2 = uti.check_accuracy(df_norm)
        per_ID_hist, per_correct_hist, per_top2_hist, correctID_hist, correcttop2_hist = uti.check_accuracy(df_hist)
        per_ID_trans, per_correct_trans, per_top2_trans, correctID_trans, correcttop2_trans = uti.check_accuracy(df_trans)

        cost_xyz, cost_rgb = uti.calc_costs(df_atlas, atlas.sigma, df_norm)
        cost_xyz_hist, cost_rgb_hist = uti.calc_costs(df_atlas, atlas.sigma, df_hist)
        cost_xyz_trans, cost_rgb_trans = uti.calc_costs(df_atlas, atlas.sigma, df_trans)

        labels.append(folder[:-4])
        per_IDs.append(per_ID)
        per_corrects.append(per_correct)
        per_corr_hist.append(per_correct_hist)
        per_corr_trans.append(per_correct_trans)
        cost.append([cost_xyz, cost_rgb])
        cost_hist.append([cost_xyz_hist, cost_rgb_hist])
        cost_trans.append([cost_xyz_trans, cost_rgb_trans])

    cost = np.asarray(cost)
    cost_hist = np.asarray(cost_hist)
    cost_trans = np.asarray(cost_trans)

    fig, ax = plt.subplots(2,1)

    x = np.arange(len(labels))
    width = 0.6

    ax[0].bar(x-width/3, per_corrects, width = width/3, label ='Percent correct original data', color= 'purple', edgecolor='black')
    ax[0].bar(x, per_corr_hist, width =width/3, label= 'Percent correct histogram eq data', edgecolor='black')
    ax[0].bar(x+width/3, per_corr_trans, width =width/3, label= 'Percent correct histogram matched data', edgecolor='black')
    ax[0].set_ylabel('Percent accuracy')
    ax[0].legend(loc='upper left', ncol=3)
    ax[0].set_title('Comparing assignment accuracy for different data processing approaches')

    ax[0].set_xticks(x)
    ax[0].set_xticklabels(labels)
    ax[0].tick_params(labelrotation=45)

    ax[1].bar(x-width/3, cost[:,0], width = width/3, label ='xyz alignment', color= 'purple', edgecolor='black')
    ax[1].bar(x, cost_hist[:,0], width =width/3, color= 'purple', edgecolor='black')
    ax[1].bar(x+width/3, cost_trans[:,0], width =width/3, color= 'purple', edgecolor='black')
    ax[1].bar(x-width/3, cost[:,1], bottom= cost[:,0], width = width/3, label ='rgb alignment', color = 'pink', edgecolor='black')
    ax[1].bar(x, cost_hist[:,1], bottom= cost_hist[:,0], width =width/3, color = 'pink', edgecolor='black')
    ax[1].bar(x+width/3, cost_trans[:,1], bottom = cost_trans[:,0], width =width/3, color = 'pink', edgecolor='black')
    ax[1].set_ylabel('Alignment cost')
    ax[1].set_title('Comparing alignment cost for different data processing approaches')
    ax[1].legend(loc='upper left', ncol=3)

    ax[1].set_xticks(x)
    ax[1].set_xticklabels(labels)
    ax[1].tick_params(labelrotation=45)

    plt.tight_layout()

    plt.show()

comp_hist_trans()

In [ ]:
%matplotlib qt

def comp_hist_trans():

    labels = []
    per_IDs = []
    per_corrects = []
    per_corr_hist = []
    per_corr_trans = []
    cost = []
    cost_hist = []
    cost_trans = []

    for folder in os.listdir('data/NP_FOCO_cropped'):
        if not folder[0:2] == '20':
            continue 

        df_norm = f.proc_FOCO('data/NP_FOCO_median/'+folder)
        df_hist = f.proc_FOCO('data/NP_FOCO_hist_med/'+folder)
        df_trans = f.proc_FOCO('data/NP_FOCO_trans_med/'+folder)

        per_ID, per_correct, per_top2, correctID, correcttop2 = uti.check_accuracy(df_norm)
        per_ID_hist, per_correct_hist, per_top2_hist, correctID_hist, correcttop2_hist = uti.check_accuracy(df_hist)
        per_ID_trans, per_correct_trans, per_top2_trans, correctID_trans, correcttop2_trans = uti.check_accuracy(df_trans)

        cost_xyz, cost_rgb = uti.calc_costs(df_atlas, atlas.sigma, df_norm)
        cost_xyz_hist, cost_rgb_hist = uti.calc_costs(df_atlas, atlas.sigma, df_hist)
        cost_xyz_trans, cost_rgb_trans = uti.calc_costs(df_atlas, atlas.sigma, df_trans)

        labels.append(folder[:-4])
        per_IDs.append(per_ID)
        per_corrects.append(per_correct)
        per_corr_hist.append(per_correct_hist)
        per_corr_trans.append(per_correct_trans)
        cost.append([cost_xyz, cost_rgb])
        cost_hist.append([cost_xyz_hist, cost_rgb_hist])
        cost_trans.append([cost_xyz_trans, cost_rgb_trans])

    cost = np.asarray(cost)
    cost_hist = np.asarray(cost_hist)
    cost_trans = np.asarray(cost_trans)

    fig, ax = plt.subplots(2,1)

    x = np.arange(len(labels))
    width = 0.6

    ax[0].bar(x-width/3, per_corrects, width = width/3, label ='Percent correct original data', color= 'purple', edgecolor='black')
    ax[0].bar(x, per_corr_hist, width =width/3, label= 'Percent correct median sample data', edgecolor='black')
    ax[0].bar(x+width/3, per_corr_trans, width =width/3, label= 'Percent correct resolution resampled data', edgecolor='black')
    ax[0].set_ylabel('Percent accuracy')
    ax[0].legend(loc='upper left', ncol=3)
    ax[0].set_title('Comparing assignment accuracy for different data processing approaches')

    ax[0].set_xticks(x)
    ax[0].set_xticklabels(labels)
    ax[0].tick_params(labelrotation=45)

    ax[1].bar(x-width/3, cost[:,0], width = width/3, label ='xyz alignment', color= 'purple', edgecolor='black')
    ax[1].bar(x, cost_hist[:,0], width =width/3, color= 'purple', edgecolor='black')
    ax[1].bar(x+width/3, cost_trans[:,0], width =width/3, color= 'purple', edgecolor='black')
    ax[1].bar(x-width/3, cost[:,1], bottom= cost[:,0], width = width/3, label ='rgb alignment', color = 'pink', edgecolor='black')
    ax[1].bar(x, cost_hist[:,1], bottom= cost_hist[:,0], width =width/3, color = 'pink', edgecolor='black')
    ax[1].bar(x+width/3, cost_trans[:,1], bottom = cost_trans[:,0], width =width/3, color = 'pink', edgecolor='black')
    ax[1].set_ylabel('Alignment cost')
    ax[1].set_title('Comparing alignment cost for different data processing approaches')
    ax[1].legend(loc='upper left', ncol=3)

    ax[1].set_xticks(x)
    ax[1].set_xticklabels(labels)
    ax[1].tick_params(labelrotation=45)

    plt.tight_layout()

    plt.show()

comp_hist_trans()

In [7]:
%matplotlib qt

def comp_two_approach(folder1, folder2):

    labels = []

    per_accs = []

    per_accs_2 = []
    
    for fold in os.listdir(folder1):
        if not fold[0:2] == '20':
            continue 

        df = f.proc_FOCO(folder1 +'/'+fold)
        df_2 = f.proc_FOCO(folder2 +'/'+fold)
        
        IDd, per_ID, per_acc = uti.get_cumul_acc(df)
        ID2, per_ID_2, per_acc_2 = uti.get_cumul_acc(df_2)

        labels.append(fold)
        per_accs.append(per_acc)
        per_accs_2.append(per_acc_2)

    per_accs = np.asarray(per_accs)
    per_accs_2 = np.asarray(per_accs_2)

    print(per_accs.shape)

    fig, ax = plt.subplots()

    x = np.arange(len(labels))
    width = 0.6

    ax.bar(x-width/4, per_accs[:,0], width = width/2, label ='Original data', edgecolor='black')
    ax.bar(x+width/4, per_accs_2[:,0], width =width/2, label= 'Resampled data', edgecolor='black')
    ax.set_ylabel('Percent accuracy')
    ax.set_ylim((0,1))
    ax.legend(loc='upper left', ncol=4)
    ax.set_title('Comparing accuracy between data processing approaches')

    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.tick_params(labelrotation=45)

    plt.tight_layout()

    plt.show()
    

comp_two_approach('data/NP_FOCO_median', 'data/NP_FOCO_resample')

(10, 4)


In [16]:
df = f.proc_FOCO('data/NP_FOCO_cropped/2022-04-26-w01-NP1')

IDd = df.loc[pd.notna(df['ID']),:]
corr1 = df.loc[df['ID']==df['autoID_1']]
corr2 = df.loc[df['ID']==df['autoID_2']]
corr3 = df.loc[df['ID']==df['autoID_3']]
corr4 = df.loc[df['ID']==df['autoID_4']]

print(len(corr1.index))
    
corr_cum_2 = pd.concat([corr1,corr2]).drop_duplicates().reset_index(drop=True)
corr_cum_3 = pd.concat([corr_cum_2,corr3]).drop_duplicates().reset_index(drop=True)
corr_cum_4 = pd.concat([corr_cum_3,corr4]).drop_duplicates().reset_index(drop=True)

per_ID = len(IDd.index)/len(df.index)
per_corr_1 = len(corr1.index)/len(IDd.index)
per_corr_2 = len(corr_cum_2.index)/len(IDd.index)
per_corr_3 = len(corr_cum_3.index)/len(IDd.index)
per_corr_4 = len(corr_cum_4.index)/len(IDd.index)


16
[[0.99998747]
 [0.84882033]
 [0.99953913]
 [0.70567437]
 [0.93178113]
 [0.98499719]
 [0.82028976]
 [0.97711951]
 [0.51119017]
 [0.95150829]
 [0.94349955]
 [0.80107247]
 [0.99751998]
 [0.76135228]
 [0.99997863]
 [0.99350924]]
5


ValueError: operands could not be broadcast together with shapes (16,1) (5,1) 

### Convert mat to tiff file

In [7]:
lin_file = sio.loadmat('data/NP_paper/interpolated/7_YAaLR_linear.mat')
cub_file = sio.loadmat('data/NP_paper/interpolated/7_YAaLR_cubic.mat')
spli_file = sio.loadmat('data/NP_paper/interpolated/7_YAaLR_spline.mat')
norm_file = sio.loadmat('data/NP_paper/all/7_YAaLR.mat')

lin_im = np.transpose(lin_file['vq'],(2,3,0,1))
cub_im = np.transpose(cub_file['vq'],(2,3,0,1))
spli_im = np.transpose(spli_file['vq'],(2,3,0,1))
norm_im = np.transpose(norm_file['data'],(2,3,0,1))

tifffile.imwrite('data/NP_paper/interpolated/7_YAaLR_linear.ome.tif', lin_im, imagej = True)
tifffile.imwrite('data/NP_paper/interpolated/7_YAaLR_cubic.ome.tif', cub_im, imagej = True)
tifffile.imwrite('data/NP_paper/interpolated/7_YAaLR_spline.ome.tif', spli_im, imagej = True)
tifffile.imwrite('data/NP_paper/interpolated/7_YAaLR_orig.ome.tif', norm_im, imagej = True)

In [43]:
imfile = sio.loadmat('data/NP_FOCO_hist/2021-12-03-w00-NP1/hist_equal_image.mat')
im = np.transpose(imfile['Hist_RGBW'],(2,3,0,1))
im = im.astype('uint16')
print(im.dtype)

tifffile.imwrite('data/NP_FOCO_hist/2021-12-03-w00-NP1/hist_eq_image.ome.tif', im, imagej = True)

uint16


In [24]:
imfile = sio.loadmat('data/NP_FOCO_trans/2022-02-12-w01-NP1/trans_data.mat')
print(imfile['trans_data'].shape)
im = np.transpose(imfile['trans_data'],(2,3,0,1))
im = im.astype('uint16')
print(im.dtype)

tifffile.imwrite('data/NP_FOCO_hist/2022-02-12-w01-NP1/trans_data.ome.tif', im, imagej = True)

(1, 1)


ValueError: axes don't match array

In [20]:
for fold in os.listdir('data/NP_FOCO_cropped'):
    if not fold[0:2] == '20':
        continue 

    df = f.proc_FOCO('data/NP_FOCO_cropped/'+fold)
    
    IDd, per_ID, per_acc = uti.get_cumul_acc(df)

    print(fold)
    print(per_ID)
    print(per_acc[0])

2022-04-26-w00-NP1
0.2111111111111111
0.07894736842105263
2022-04-26-w01-NP1
0.2
0.1111111111111111
2021-12-03-w00-NP1
0.3023255813953488
0.6346153846153846
2022-02-22-w04-NP1
0.25287356321839083
0.6818181818181818
2022-04-01-w00-NP1
0.26506024096385544
0.45454545454545453
2022-03-05-w00-NP1
0.27976190476190477
0.5106382978723404
2022-02-12-w00-NP1
0.3028571428571429
0.2830188679245283
2022-02-12-w01-NP1
0.3016759776536313
0.4444444444444444
2022-01-22-w04-NP1
0.2823529411764706
0.10416666666666667
2022-02-11-w03-NP1
0.24404761904761904
0.12195121951219512


In [53]:
for folder in os.listdir('data/NP_FOCO_resample'):
    if folder == '.DS_Store':
        continue
    imfile = sio.loadmat('data/NP_FOCO_resample/'+ folder + '/resample_image.mat')
    im = np.transpose(imfile['data'],(2,3,0,1))
    im = im.astype('uint16')
    print(im.dtype)

    tifffile.imwrite('data/NP_FOCO_resample/'+ folder + '/resample_image.tif', im, imagej = True)

uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
